In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Store csv created in part one into a DataFrame
file1 = pd.read_csv("output_data/cities.csv")
file1

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?')

In [6]:
# Heatmap of humidity


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down cities that fit criteria and drop any results with null values


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,rikitea,-23.12,-134.97,71.01,67,0,4.90,PF,1569425987
31,31,barrow,-38.31,-60.23,71.55,40,0,6.35,AR,1569426234
88,88,niteroi,-29.95,-51.16,79.00,53,0,3.36,BR,1569426245
160,160,shache,38.42,77.24,75.00,51,0,7.65,CN,1569426258
303,303,namibe,-15.19,12.15,70.47,88,0,9.48,AO,1569426288


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates


,City,Country,Lat,Lng,Hotel Name
1,rikitea,PF,-23.12,-134.97,
31,barrow,AR,-38.31,-60.23,
88,niteroi,BR,-29.95,-51.16,
160,shache,CN,38.42,77.24,
303,namibe,AO,-15.19,12.15,


In [9]:
# Set parameters to search for a hotel


# Iterate through 


Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
1,rikitea,PF,-23.12,-134.97,Pension Maro'i
31,barrow,AR,-38.31,-60.23,
88,niteroi,BR,-29.95,-51.16,Hotel Express Canoas
160,shache,CN,38.42,77.24,Delong Hotel
303,namibe,AO,-15.19,12.15,Chik-Chik Namibe


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map


# Display figure


Figure(layout=FigureLayout(height='420px'))